# COVID-19 Dashboard

This dashboard was created using [Public Health England](https://www.gov.uk/government/organisations/public-health-england) (PHE) data to present up-to-date vaccination dose (Pfizer/BioNTech or Oxford/AstraZeneca) statistics. 

Tables and graphs were created using pandas, matplotlib, numpy and ipywidgets libraries. 

This dashboard filters [COVID-19 API data](https://coronavirus.data.gov.uk/details/developers-guide/main-api) in the following ways:

* First, Second and Third Doses (cumulative totals)
* NHS Region of Residence (nation = England)
* Age Group (all age groups)
* Gender (all genders)
* Ethnic Groups (all ethnic groups)

In [17]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from uk_covid19 import Cov19API
import json 

In [18]:
%matplotlib inline
plt.rcParams['figure.dpi'] = 100

In [19]:
jsondata={}

In [24]:
filters = [
    'areaType=nation',
    'areaName=England'
]
structure = {
    "date": "date",
    "firstdose": "cumPeopleVaccinatedFirstDoseByPublishDate",
    "seconddose": "cumPeopleVaccinatedSecondDoseByPublishDate",
    "thirddose": "cumPeopleVaccinatedThirdDoseByPublishDate"
}

In [25]:
api = Cov19API(filters=filters, structure=structure)

doseseries=api.get_json()

In [26]:
with open("doseseries.json", "wt") as OUTF:
    json.dump(doseseries, OUTF)

In [27]:
with open("doseseries.json", "rt") as INFILE:
    data=json.load(INFILE)

In [28]:
datalist=data['data']

In [29]:
dates=[dictionary['date'] for dictionary in datalist ]
dates.sort()

In [30]:
def parse_date(datestring):
    return pd.to_datetime(datestring, format="%Y-%m-%d")

In [31]:
startdate=parse_date(dates[0])
enddate=parse_date(dates[-1])

In [32]:
index=pd.date_range(startdate, enddate, freq='D')
doseseriesdf=pd.DataFrame(index=index, columns=['firstdose', 'seconddose', 'thirddose'])

In [33]:
for entry in datalist: 
    date=parse_date(entry['date'])
    for column in ['firstdose', 'seconddose', 'thirddose']:
        if pd.isna(doseseriesdf.loc[date, column]): 
            value= float(entry[column]) if entry[column]!=None else 0.0
            doseseriesdf.loc[date, column]=value
            
doseseriesdf.fillna(0.0, inplace=True)
            
doseseriesdf

,firstdose,seconddose,thirddose
2021-01-10,1959151.0,374613.0,0.0
2021-01-11,2080280.0,393925.0,0.0
2021-01-12,2254556.0,407294.0,0.0
2021-01-13,2494371.0,415656.0,0.0
2021-01-14,2769164.0,420510.0,0.0
...,...,...,...
2021-12-02,42851727.0,38964371.0,0.0
2021-12-03,42872533.0,38990840.0,0.0
2021-12-04,42897322.0,39025116.0,0.0
2021-12-05,42913330.0,39045988.0,0.0


Figure 1: This table presents the cumulative number of people, of all ages, who have received a first, second & third dose of the COVID-19 vaccination, across England in a 7-day rolling average.

In [34]:
series=wdg.SelectMultiple(
    options=['firstdose', 'seconddose', 'thirddose'],
    value=['firstdose', 'seconddose', 'thirddose'],
    rows=3,
    description='Doses:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)

controls=wdg.HBox([series, scale])

def doseseries_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        doseseriesdf[list(gcols)].plot(logy=logscale)
        plt.show() 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

graph=wdg.interactive_output(doseseries_graph, {'gcols': series, 'gscale': scale})
display(controls, graph)

Output()

Figure 2: The line graph above interactively, presents the trajectory of cumulative number of first, second and third doses received across England. 

In [35]:
def access_api(button):
    filters = [
        'areaType=nation'
    ]
    structure = {
        "date": "date",
        "firstdose": "cumPeopleVaccinatedFirstDoseByPublishDate",
        "seconddose": "cumPeopleVaccinatedSecondDoseByPublishDate",
        "thirddose": "cumPeopleVaccinatedThirdDoseByPublishDate"
    }

    api = Cov19API(filters=filters, structure=structure)
    doseseries=api.get_json()
    apibutton.icon="check"
    apibutton.disabled=False


apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='Success', 
    tooltip='Click to download current Public Health England data',
    icon='fa-redo' 
)


apibutton.on_click(access_api)
display(apibutton)

Button(button_style='success', description='Refresh data', icon='redo', style=ButtonStyle(), tooltip='Click to…

Press the button above to refresh the data in the dashboard and consequently the tables and graphs contained within it. 